In [ ]:
#import statements, hyper params, models
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from matplotlib.pyplot import figure

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import warnings
warnings.filterwarnings('ignore') 
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from numpy import random
from random import randrange
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import time
    
THRESH_HOLD=5e-4            #MSE value when we consider the second model to have "cracked" the first
EPOCHS = 25                 #number of passes of whole data
BATCH_SIZE =  64            #size of data going through at once
LEARNING_RATE = 0.01        #how much we shift parameters during back prop
NEURONS=5                   #number of nodes in each layer
INPUTS=5                     #number of input vars
XSIZE=  577                 #size of each data set generated
BAD_LOSS_PASSES=25          #number of times we have small loss until we stop code
NO_TIME_INCREASES_STOP=10        #number of interations with no time increase until we stop code
TESTMODULE_LAYERS=2         #number of layers in cracking module
GROUND_TRUTH_LAYERS=10       #number of layers in ground thruth models
LINCOMBO=True               #wether to take best new time or linear combination
activation=nn.Sigmoid()     #activation     
def get_Xvals():
    return (np.random.uniform( size=(XSIZE, INPUTS))-0.5)*np.sqrt(12)
    
def getdata_XY():
    X=(np.random.uniform( size=(XSIZE, INPUTS))-0.5)*np.sqrt(12)
    test_data = testData(torch.FloatTensor(X))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist =[]
    
    
    GroundTruth.eval()
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = GroundTruth(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return X, y_truthlist

def get_yvals(model, X_data):
    model.eval()
    test_data = testData(torch.FloatTensor(X_data))
    test_loader = DataLoader(dataset=test_data, batch_size=1)
    y_truthlist=[]
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_test_pred = model(X_batch)
            y_test_pred = torch.sigmoid(y_test_pred)
            y_truthlist.append(y_test_pred.cpu().numpy())


    y_truthlist = [a.squeeze().tolist() for a in y_truthlist]
    return y_truthlist

#Data class
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

#Data loaders (to look at data) and deal with data (training and testing)
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

#trains a network, returns time spent training 
def trainnetwork(network, train_loader, optimizer, Time=True):
    t0=time.time()
    network.to(device)
    criterion = nn.MSELoss()


    network.train()
    for e in range(1, EPOCHS+1):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = network(X_batch)
            y_pred = torch.sigmoid(y_pred)
            loss = criterion(y_pred, y_batch.unsqueeze(1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()        
            
    t1=time.time()
    return(t1-t0)

#returns next ground truth model
#model obtained by taking average of models with better times than previous model 
def get_new_GroundTruth_lincombo(model_list, time_list, time):
    better_model_list=[]
    for (i, model) in zip(time_list, model_list):
        if i>time:
            better_model_list.append(model)
    
    
    newGroundTruth=GroundTruth.get_copy()
    
    layer_weight=torch.tensor([[0]*INPUTS]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_input.weight/(len(better_model_list))
        
    layer_weight.requires_grad_()    
    newGroundTruth.layer_input.weight=torch.nn.Parameter(layer_weight)
    
    
    for i in range(GROUND_TRUTH_LAYERS):
        layer_weight=torch.tensor([[0]*NEURONS]*NEURONS, requires_grad=False)
        for bettermodel in better_model_list:
            layer_weight=layer_weight+bettermodel.layers[i].weight/(len(better_model_list))
        layer_weight.requires_grad_()
        newGroundTruth.layers[i].weight=torch.nn.Parameter(layer_weight)
        
    layer_weight=torch.tensor([0]*NEURONS, requires_grad=False)
    for i in better_model_list:
        layer_weight=layer_weight+i.layer_output.weight/(len(better_model_list))
    layer_weight.requires_grad_()   
    newGroundTruth.layer_output.weight=torch.nn.Parameter(layer_weight)

    return newGroundTruth
        
#checks accuracy of model
def check_threshhold(network, test_loader, truth):
    total_loss=0
    network.eval()
    with torch.no_grad():
        for (X_batch, y) in zip(test_loader, truth):
            X_batch = X_batch.to(device)
            y_test_pred = network(X_batch)
            y_pred_tag = torch.sigmoid(y_test_pred)
            total_loss=total_loss+((y_pred_tag.cpu().numpy()-y))**2
    
    total_loss=total_loss/len(truth)

    if total_loss[0][0]<THRESH_HOLD:
        return True, total_loss[0][0]
    else:
        return False, total_loss[0][0]
                       
def select_node():
    layer=randrange(GROUND_TRUTH_LAYERS+2)
    if layer==0:
        node=[randrange(INPUTS), randrange(NEURONS)]
    elif layer==GROUND_TRUTH_LAYERS+1:
         node=randrange(NEURONS)
    else:
        node=[randrange(NEURONS), randrange(NEURONS)]
    return [layer, node]
       
class arbitraryLayerModel(nn.Module):
    def __init__(self, num_layers):
        super(arbitraryLayerModel, self).__init__()
        self.layer_input=nn.Linear(INPUTS, NEURONS)
        self.layers=nn.ModuleList([nn.Linear(NEURONS, NEURONS) for i in range(num_layers)])
        
        
        self.batchnorms=nn.ModuleList([nn.BatchNorm1d(NEURONS) for i in range(num_layers)])
        self.batchnorm_input=nn.BatchNorm1d(NEURONS)
        
        self.layer_output=nn.Linear(NEURONS,1)
        self.dropout = nn.Dropout(p=0.1)
        
    def get_copy(self):
        copied_model=arbitraryLayerModel(len(self.layers))
        copied_model.layers=nn.ModuleList()
        for i in self.layers:
            copied_model.layers.append(copy.deepcopy(i))
            
        copied_model.batchnorms=nn.ModuleList()
        for i in self.batchnorms:
            copied_model.batchnorms.append(copy.deepcopy(i))
        copied_model.layer_input=copy.deepcopy(self.layer_input)
        copied_model.layer_output=copy.deepcopy(self.layer_output)
        copied_model.batchnorm_input=copy.deepcopy(self.batchnorm_input)

        return copied_model
            
     
    def forward(self, inputs):
        x=activation(self.batchnorm_input(self.layer_input(inputs)))
        x=self.dropout(x)
        
        for (layer, batchnorm) in zip(self.layers, self.batchnorms):
            x=activation(batchnorm(layer(inputs)))
            x=self.dropout(x)
                
                
        x=self.layer_output(x)
        return(x)

    
    
    def change_node_model(self, node):
        changed_node=self.get_copy()
        
        weightchange=np.random.normal(0,1)      
        
        if node[0]==0:
            zeros=[[0]*INPUTS]*NEURONS
            zeros[node[1][0]][node[1][1]]=weightchange
            newweight=self.layer_input.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_input.weight=torch.nn.Parameter(newweight)    
            return changed_node
            
            
        
        if node[0]==GROUND_TRUTH_LAYERS+1:
            zeros=[0]*(NEURONS-1)
            zeros.insert(node[1],weightchange)
            newweight=self.layer_output.weight.requires_grad_(False)+torch.tensor(zeros,requires_grad=False)
            newweight.requires_grad_()
            changed_node.layer_output.weight=torch.nn.Parameter(newweight) 
            
            
        
        else:
            zeros = [[0.0]*NEURONS]*NEURONS
            for layer, i in enumerate(changed_node.layers):
                if i==node[0]+1:
                    zeros[node[1][0]][node[1][1]]=weightchange
                    newweight=layer.weight.requires_grad_(False)+torch.tensor(weightchange,requires_grad=False)
                    newweight.requires_grad_()
                    layer.weight=torch.nn.Parameter(newweight)
                    
        

        return changed_node

   
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   

    



In [ ]:
#linear combo 1 weight guassian weight
       
#groundtruth model  
GroundTruth = arbitraryLayerModel(GROUND_TRUTH_LAYERS)
possible_infinite_time=arbitraryLayerModel(GROUND_TRUTH_LAYERS)



firstmodeltrained=arbitraryLayerModel(TESTMODULE_LAYERS)
test_model_optimizer = optim.Adam(firstmodeltrained.parameters(), lr=LEARNING_RATE)
accepted=False
current_time=0
data_list=[]


while((not accepted)):
    X, y=getdata_XY()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
    test_data = testData(torch.FloatTensor(X_test))
    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=False)                      
    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)
    data_list.append(X)
    current_time=current_time+trainnetwork(firstmodeltrained, train_loader,test_model_optimizer)
    accepted=check_threshhold(firstmodeltrained, test_loader, y_test)
    
print("First time: ", current_time)
    
#iterates
interationcounter=1
#number of iterates in a row with no time increase
no_increasecounter=0
time_increase=True
#number of training interations where loss doesnt change
loss_difference_counter=0
stop_itterations=False
while(time_increase):
    node_list=[]
    time_list=[]
    model_list=[]
    for i in range(0, 10):
        node_list.append(select_node())
    
    test_model_list=[arbitraryLayerModel(TESTMODULE_LAYERS) for i in range(10)]
    for i in range(0, 10):
        total_time=0
        infinite_time_counter=0
        prospective_GroundTruth=GroundTruth.change_node_model(node_list[i])

        model_list.append(prospective_GroundTruth)

        for j, test_model in enumerate(test_model_list):
            current_loss=0
            previous_loss=0
            train_testmodel=test_model.get_copy()
            test_model_optimizer = optim.Adam(train_testmodel.parameters(), lr=LEARNING_RATE)
            data_counter=0
            test_time=0
            inital_data_length=len(data_list)
            accepted=False
            loss_difference_counter=0
            Model_beaten=False

            while(not accepted): 
                #use previously generated data
                if data_counter<inital_data_length:
                    X=data_list[data_counter]
                    y=get_yvals(prospective_GroundTruth, X)
                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=False)




                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    previous_loss=current_loss
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=previous_loss-current_loss
                    data_counter=data_counter+1
                #generate new data if needed
                else:                          
                    X=get_Xvals()
                    data_list.append(X)
                    y=get_yvals(prospective_GroundTruth, X)



                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
                    train_data= trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train))
                    test_data = testData(torch.FloatTensor(X_test))
                    train_loader= DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)                      
                    test_loader = DataLoader(dataset=test_data, batch_size=1)

                    previous_loss=current_loss
                    test_time=test_time+trainnetwork(train_testmodel, train_loader,test_model_optimizer)
                    accepted, current_loss=check_threshhold(train_testmodel, test_loader, y_test)
                    loss_diff=previous_loss-current_loss


                #checks if loss difference is small
                
                if abs(loss_diff)<(THRESH_HOLD/10):
                    loss_difference_counter=loss_difference_counter+1
                    if loss_difference_counter>BAD_LOSS_PASSES:
                        infinite_time_counter=infinite_time_counter+1
                        print("Test model beaten. Total beaten by prospective ground truth: ", infinite_time_counter)
                        Model_beaten=True
                    elif loss_difference_counter>5:
                        print(f'Number no loss change {loss_difference_counter}: | Current time: {test_time:.3f} | Loss difference: {abs(loss_diff):.4} | Loss: {current_loss:.4}')
 
            
                if Model_beaten:
                    break


            total_time=total_time+test_time

    
    
        print(f'Iteration: {interationcounter} | Ground truth number: {i+1} | Average time:  {total_time/10:.3}')

        if infinite_time_counter==10:
            print("Potential unbreakable found")
            possible_infinite_time=prospective_GroundTruth.get_copy()
            break
        time_list.append(total_time/(10))

        
           
    
    if infinite_time_counter==10:
        break
    max_new_time=max(time_list)
    
    
    
    #prints iteration data
    print(f'Iteration {interationcounter}: | Previous time: {current_time:.3f} | Best new time: {max_new_time:.3f} | Difference: {max_new_time-current_time:.3f}')
    interationcounter=interationcounter+1
    
    
    
     
    
    
    time_increase=max_new_time-current_time
    
    
    
    #changes ground truth if better time found
    if max_new_time>current_time:
        if LINCOMBO:
            GroundTruth=get_new_GroundTruth_lincombo(model_list, time_list, current_time)
        else:
            maxindex=time_list.index(max(time_list))
            GroundTruth=model_list[maxindex].get_copy()
        time_increase=True
        previous_time=current_time
        current_time=max_new_time
        no_increasecounter=0
        
    else:
        no_increasecounter=no_increasecounter+1

    #if no better ground truth found for 10 iterations, stops code                 
    if no_increasecounter==NO_TIME_INCREASES_STOP:
        print("No increase for ", NO_TIME_INCREASES_STOP,  " iterations")
    #    time_increase=False
 
      

                    